This scrapes 2010 zip code data from zip-codes.com.  

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
#create list of zipcodes from txt file with zips
zips = open('chosen_zips.txt', 'r').readlines()
zips = [z.replace('\n', '') for z in zips]
zipcode = zips

In [4]:
df_main = pd.DataFrame()
i = 0
num = len(zipcode)
for i in range(num):
    
    baseURL = 'https://www.zip-codes.com/zip-code/'+zipcode[i]+'/zip-code-'+zipcode[i]+'-2010-census.asp'


    #calledURL = baseURL % (zipcode, zipcode)
    source = requests.get(baseURL)
    soup = BeautifulSoup(source.text, 'lxml')

    #get data by looking for each tr
    #turns it all into a list, and we're using list comprehension
    data = []
    for tr in soup.find_all('tr'):
        values = [td.text for td in tr.find_all('td')]
        data.append(values)
        #the info we want is within a list of lists
        #just searching for one part of the sublist will not work
        #['Average household size of renter-occupied units', '2.35', '( X )'] in data

    #there has to be a better way to do this
    #but for now I have to create a sublist for every search column
    basic = [[],[],[],[],[],[],[],[],[],[]]
    search = ['Total 2010 Census Population for ZIP Code '+zipcode[i],
              'Total households for ZIP Code '+zipcode[i],
              'Average household size of ZIP Code '+zipcode[i], 
              'Median age (years)',
              '18 years and over',
              '65 years and over',
         ]


    i = 0
    num = len(search)
    for i in range(num):
        for sublist in data:
            if sublist[0] == search[i]:
                basic[i].append(sublist[1])
        i = i +1


    #because we only need the 0 element in each sublist, drop the rest
    #this gives us the 2010 values for this zipcode
    df = pd.DataFrame(data=basic)
    df_tr = df.transpose()
    df_tr = df_tr[:1]
    

    df_main = df_main.append(df_tr)



#now lets add the zip code and year columns
#starting with year cause it's easy
df = df_main
df['year'] = '2010'
df['zipcode'] = zipcode

#drop the empty columns
df = df.drop(df.columns[[6,7,8,9]], axis = 1)

df.columns = ['pop', 'hhold', 'avg_hhold', 'med_age', '18plus', '65plus', 'year', 'zipcode']

name_df = pd.read_excel("town_names.xlsx", dtype={'zipcode': object})
df = df.set_index('zipcode').join(name_df.set_index('zipcode'))



#move some columns around so it will be easier to read in excel
#remember zipcode is the index not a column
df = df[['territory', 'town', 'county', 'year', 'pop', 
         'hhold', 'avg_hhold', 'med_age', '18plus', '65plus' ]]



thousand seperator comma removal

In [6]:
df['pop'] = df['pop'].str.replace(',','').astype(int)
df['hhold'] = df['hhold'].str.replace(',','').astype(int)
df['18plus'] = df['18plus'].str.replace(',','').astype(int)
df['65plus'] = df['65plus'].str.replace(',','').astype(int)

In [7]:
df.to_excel('./excel/2010_scrape.xlsx')